## Change voxel spacing / Create isotropic volume
This code changes voxel spacing of a 3D volume. It can also be used to generate isotropic volume.

In [1]:
import numpy as np
import os
import pydicom
from scipy.interpolate import RegularGridInterpolator

### Function to read DICOM files

More description at https://github.com/mrinal054/generate_3d_volume_from_DICOM_images.git to read DICOM files

In [2]:
def get_dcm(dcm_path):
    dicom_img = []
    file_names = []
    # Get DICOM image names
    for dir_name, sub_dir_list, file_list in os.walk(dcm_path):
        for file_name in file_list:
            if ".dcm" in file_name.lower():  # check if it is dcm file 
                file_names.append(file_name)
                dicom_img.append(os.path.join(dir_name,file_name))
    
    # Get ref file
    ref = pydicom.dcmread(dicom_img[0])    
    # Get the spacing
    pix_dim = (int(ref.Rows), int(ref.Columns), len(dicom_img))
    # Get spacing values (in mm)
    pix_spacing = (float(ref.PixelSpacing[0]), float(ref.PixelSpacing[1]), float(ref.SliceThickness))    
    # Get the data
    dcm_data = np.zeros(pix_dim, dtype=ref.pixel_array.dtype)   

    # Check if the DICOM file has 'SliceLocation' attribute. It if it has, then store them
    # according to the slice location. Otherwise, store them chronologically. 
    if hasattr(pydicom.dcmread(dicom_img[0]), 'SliceLocation'):               
        # Sort file names according to the slice location
        sliceLocation = [pydicom.dcmread(dicom_img[i]).SliceLocation for i in range(len(dicom_img))]
        ps = np.argsort(sliceLocation) #ps: position of the sorted slice locations
        sorted_file_names = [file_names[ps[i]] for i in range(len(ps))]
        print('DICOM sorted by slice location')          
        
        # loop through all the DICOM files
        idx = 0
        for file in sorted_file_names:       
            # read the file
            ds = pydicom.dcmread(os.path.join(dir_name,file))
            # store the raw image data
            dcm_data[:, :, idx] = ds.pixel_array
            idx += 1
    else:
        print('No slice location. DICOM sorted by name')

        # loop through all the DICOM files
        for file in dicom_img:
            # read the file
            ds = pydicom.dcmread(os.path.join(dir_name,file))
            # store the raw image data
            dcm_data[:, :, dicom_img.index(file)] = ds.pixel_array                              
    
    # Get the min and max intensity
    intensity = (dcm_data.min(), dcm_data.max())
        
    return dcm_data, pix_spacing, intensity

### Function to change voxel spacing

* Function `change_voxel_spacing` changes voxel spacing

* Input parameters: <br>
    `D`: Original data <br>
    `org_spacing`: Original voxel spacing <br>
    `new_spacing`: New voxel spacing defined by the user
    
* Output:
    Returns volume with new spacing in `int16` format

Author @ Mrinal Kanti Dhar

In [3]:
def change_voxel_spacing(D, org_spacing, new_spacing):
    # Get new volume size
    D = np.squeeze(D)
    sz_org = D.shape #size of original 3D data
    ratio = org_spacing/new_spacing
    sz_new = np.ceil((D.shape-np.array([1]))*ratio)+np.array([1]) #size of new 3D data
    sz_new = sz_new.astype('int16')

    # Generate voxel locations for the original data
    Xa = np.arange(0, sz_org[0], 1)*org_spacing[0] 
    Ya = np.arange(0, sz_org[1], 1)*org_spacing[1]
    Za = np.arange(0, sz_org[2], 1)*org_spacing[2]
    
    # Generate voxel locations for the new data
    Xb, Yb, Zb = np.meshgrid(np.arange(0, sz_new[0], 1)*new_spacing[0],
                             np.arange(0, sz_new[1], 1)*new_spacing[1],
                             np.arange(0, sz_new[2], 1)*new_spacing[2])
    
    # Do interpolation
    fn = RegularGridInterpolator((Ya,Xa,Za), D, bounds_error=False)
    
    Xb_f, Yb_f, Zb_f = Xb.flatten(), Yb.flatten(), Zb.flatten() #flatten data

    interpPoints = np.array([Yb_f, Xb_f,  Zb_f]) 
    
    D_new = fn(interpPoints.T)
    D_new = np.reshape(D_new, Xb.shape)
    D_new = D_new.astype('int16') #saving in int16 format    
    
    return D_new

#### Read DICOM files

In [4]:
dcm_loc = 'F:\dicom'
data, pix_spacing_org, intensity_org = get_dcm(dcm_loc)
print('Original dimension: ', data.shape, 
      '\nOriginal voxel spacing: ', pix_spacing_org, 
      '\nOriginal intensity range: ', intensity_org)

DICOM sorted by slice location
Original dimension:  (480, 480, 320) 
Original voxel spacing:  (0.263602, 0.263602, 0.2515) 
Original intensity range:  (-1100, 3899)


#### Change voxel spacing

In [5]:
pix_spacing_new = np.array([0.4, 0.4, 0.4]) #set new spacing 

data_new = change_voxel_spacing(data, pix_spacing_org, pix_spacing_new)

pix_dim_new = data_new.shape
print('New dimension: ', pix_dim_new, 
      '\nNew voxel spacing: ', pix_spacing_new, 
      '\nNew intensity range: ', data_new.min(), data_new.max())

New dimension:  (317, 317, 202) 
New voxel spacing:  [0.4 0.4 0.4] 
New intensity range:  -1100 3899
